### rozwiązanie problemu komiwojażera
## Algorytm Genetyczny
#### Komosa Maciej, Informatyka i Ekonometria, studia stacjonarne, 3 rok

Algorytm genetyczny polega na wygenerowaniu pewnego zbioru potencjalnych rozwiązań danego problemu, a następnie ocenienie rozwiązań daną funkcją dopasowania, wybraniu *osobników* o najlepszych wynikach oraz *krzyżowaniu* ich celem uzyskania nowych *pokoleń*, które będą osiągały lepsze wyniki od poprzednich. 

W tym projekcie:
- populacja początkowa generowana jest w sposób losowy
- funkcją dopasowania jest całkowita długość trasy
- są cztery metody wyboru najlepszych *osobników*:
    - turniejowa, w której *rodzice* porównywani są w parach
    - rankingowa, ... losowani są z prawdopodobieństwem zależnym od ich miejsca w rankingu
    - ruletkowa, ... prawdopodobieństwem równym *1/dopasowanie*
    - autorska, #TODO
- krzyżowanie przebiega zgodnie z jedną z dwóch metod:
    - OX: wylosowane zostają dwa *punkty przecięcia*, które dzielą *osobnika* na trzy części: środkowa przepisywana jest z rodzica pierwszego, a pierwsza i ostatnia z drugiego, przy czym w przypadku powtórzenia się którejś z liczb wybierana jest kolejna
    - autorska, #TODO
- istnieje prawdopodobieństwo mutacji, w której dwa elementy *osobnika* zamieniane są ze sobą miejscami

## importowanie podstawowych bibliotek

In [43]:
import numpy as np
import pandas as pd
from itertools import chain
from tqdm import tqdm

## wczytywanie i zapisywanie danych i generowanie początkowych kombinacji

In [44]:
# zwraca dane z pliku xlsx w postaci ndarray
def wczytajDaneXlsx(sciezkaPliku: str):
    dane = pd.read_excel(sciezkaPliku)
    dane.fillna(0, inplace=True)
    return dane.values

# zwraca ndarray wszystkich wyników populacji, [populacja][osobnik]
def archiwizujPopulacje(populacja: np.ndarray, wynikiPopulacji: np.ndarray, archiwum, iteracja):
    populacjaZWynikami = np.column_stack((populacja, wynikiPopulacji))
    archiwum[iteracja] = populacjaZWynikami
    return archiwum

# zwraca populacje początkową, ndarray n losowych permutacji punktów zawartych w danych
def generujPopulacjęPoczątkową(dane: np.ndarray, n):
    indeksy = list(range(len(dane)))
    populacja = np.zeros(n, dtype=np.ndarray)
    for i in range(n):
        populacja[i] = np.random.permutation(indeksy)
    return populacja

## obliczanie wyników wybranej populacji

In [45]:
# zwraca wyniki populacji, ndarray zawierający długości tras dla danego osobnika
def obliczWynikiPopulacji(dane: np.ndarray, populacja: np.ndarray):
    wynikiPopulacji = np.zeros(len(populacja), dtype=np.ndarray)
    for i in range(len(populacja)):
        wynikiPopulacji[i] = (obliczOdleglosc(dane, populacja[i]))
    return wynikiPopulacji

# zwraca odległość trasy danego osobnika
def obliczOdleglosc(dane: np.ndarray, osobnik):
    odleglosc = 0
    for i in range(len(osobnik)-1):
        odleglosc += dane[int(osobnik[i])][int(osobnik[i+1])]
    odleglosc += dane[int(osobnik[-1])][int(osobnik[0])]
    return odleglosc

# zwraca listę n% najlepszych wyników
def obliczProcentNajlepszych(wynikiPopulacji: np.ndarray, procent):
    wynikiPopulacji = np.argsort(wynikiPopulacji)
    wynikiPopulacji = wynikiPopulacji[:int(len(wynikiPopulacji)*procent)]
    return wynikiPopulacji

## wybieranie rodziców

In [46]:
# zwraca listę najlepszych rodziców
def wybierzNajlepszychRodziców(dane: np.ndarray, populacja: np.ndarray, metoda = 'turniejowa'):
    wynikiPopulacji = obliczWynikiPopulacji(dane, populacja)

    if metoda == 'turniejowa':
        rodzice = wybierzNajlepszychRodzicówMetodąTurniejową(populacja, wynikiPopulacji)
    elif metoda == 'ruletkowa':
        rodzice = wybierzNajlepszychRodzicówMetodąRuletkową(populacja, wynikiPopulacji)
    else:
        raise Exception('Nieznana metoda selekcji')
    return rodzice

# zwraca listę 1/n najlepszych osobników z populacji
def wybierzNajlepszychRodzicówMetodąTurniejową(populacja: np.ndarray, wynikiPopulacji: np.ndarray, n = 2):
    rodzice = np.zeros(len(populacja) // n, dtype=np.ndarray)
    for i in range(len(rodzice)):
        if wynikiPopulacji[2*i] < wynikiPopulacji[(2*i + 1) % (len(populacja))]:
            rodzice[i] = populacja[2*i]
        else:
            rodzice[i] = populacja[(2*i + 1) % (len(populacja))]
    return rodzice

# zwraca listę n% najlepszych osobników z populacji
def wybierzNajlepszychRodzicówMetodąRuletkową(populacja, wynikiPopulacji, procent = 50):
    rodzice = np.zeros(int(len(populacja) * procent / 100))
    prawdopodobienstwa = np.zeros(len(populacja))
    sumaWynikow = sum(wynikiPopulacji)
    for i in range(populacja):
        prawdopodobienstwa[i] = (wynikiPopulacji[i] / sumaWynikow)
    for i in range(int(len(populacja) * procent)):
        rodzice[i] = np.random.choice(populacja, p=prawdopodobienstwa)
    return rodzice

## krzyżowanie

In [47]:
# przyjmuje rodziców, paruje ich i zwraca dzieci
def krzyżuj(rodzice, metoda = 'OX'):
    rodziceWParach = paruj(rodzice)
    potomstwoWParach = np.zeros_like(rodziceWParach)
    if metoda == 'OX':
        krzyżujParę = krzyżujParęMetodąOX
    elif metoda == 'autorska':
        krzyżujParę = krzyżujParęMetodąAutorską
    else:
        raise Exception('Nieznana metoda krzyżowania!')
    
    for i in range(len(rodziceWParach)):
        potomstwoWParach[i] = krzyżujParę(rodziceWParach[i])

    potomstwo = np.zeros(len(rodziceWParach) * 2, dtype=np.ndarray)
    for i in range(len(potomstwoWParach)):
        potomstwo[2*i] = np.array(potomstwoWParach[i][0], dtype=int)
        potomstwo[2*i + 1] = np.array(potomstwoWParach[i][1], dtype=int)
    
    return potomstwo

# łączy rodziców w pary i zwraca listę par 
def paruj(rodzice: np.ndarray):
    rodzice_w_parach = np.zeros(len(rodzice), dtype=np.ndarray)

    for i in range(0, rodzice.shape[0] - 1):
        pair = np.array([rodzice[i], rodzice[i + 1]])
        rodzice_w_parach[i] = pair

    pair = np.array([rodzice[-1], rodzice[0]])
    rodzice_w_parach[-1] = pair

    return np.array(rodzice_w_parach)

# krzyżuje parę rodziców metodą OX, zwraca parę dzieci
def krzyżujParęMetodąOX(rodzice: np.ndarray):
    assert len(rodzice) == 2
    potomstwo = np.full(rodzice.shape, np.nan)
    punkty_przekroju = np.random.choice(range(1, len(rodzice[0]) - 1), 2, replace=False)
    punkty_przekroju.sort()
   
    for i in range(len(potomstwo)):
        potomstwo[i, punkty_przekroju[0]:punkty_przekroju[1]] = rodzice[i, punkty_przekroju[0]:punkty_przekroju[1]]
        for j in chain(range(punkty_przekroju[0]), range(punkty_przekroju[1], len(potomstwo[i]))):
            if potomstwo[i, j] == np.nan:
                potomstwo[i, j] = rodzice[(i+1)%2, j]
            else:
                k = 0 
                while rodzice[(i+2)%2, k] in potomstwo[i]:
                    k += 1
                potomstwo[i, j] = rodzice[(i+2)%2, k]
    
    return potomstwo

def krzyżujParęMetodąAutorską(rodzice):
    #todo
    return []

## mutacja

In [48]:
def mutuj(populacja: np.ndarray, prawdopodobienstwoMutacji = 0.1):
    for i in range(len(populacja)):
        if np.random.random() < prawdopodobienstwoMutacji:
            indices = np.random.choice(range(len(populacja[i])), 2, replace=False)
            populacja[i][indices[0]], populacja[i][indices[1]] = populacja[i][indices[1]], populacja[i][indices[0]]
    return populacja

## pełen przebieg algorytmu

In [49]:
def algorytm_genetyczny(dane, n=25, iteracje=100, metodaSelekcji='turniejowa',
                        metodaKrzyzowania='OX', prawdopodobienstwoMutacji=0.1):
    populacja = generujPopulacjęPoczątkową(dane, n)
    wynikiPopulacji = obliczWynikiPopulacji(dane, populacja)
    archiwum = np.zeros(iteracje, dtype=np.ndarray)
    archiwum = archiwizujPopulacje(populacja, wynikiPopulacji, archiwum, 0)
    for i in tqdm(range(0, iteracje)):
        rodzice = wybierzNajlepszychRodziców(dane=dane, populacja=populacja, metoda=metodaSelekcji)
        populacja = krzyżuj(rodzice=rodzice, metoda=metodaKrzyzowania)
        populacja = mutuj(populacja=populacja, prawdopodobienstwoMutacji=prawdopodobienstwoMutacji)
        wynikiPopulacji = obliczWynikiPopulacji(dane=dane, populacja=populacja)
        archiwum = archiwizujPopulacje(populacja=populacja, wynikiPopulacji=wynikiPopulacji,
                                        archiwum=archiwum, iteracja=i)
    
    najlepszy_wynik = None
    tabela_iteracji = pd.DataFrame()
    for i in range(len(archiwum)):
        archiwum[i] = archiwum[i][archiwum[i][:, -1].argsort()]
        tabela_iteracji[i] = archiwum[i][0]
        if najlepszy_wynik is None or archiwum[i][0][-1] < najlepszy_wynik[-1]:
            najlepszy_wynik = archiwum[i][0]
    return najlepszy_wynik, tabela_iteracji

dane = wczytajDaneXlsx('Dane_TSP_127.xlsx')
wynik, tabela = algorytm_genetyczny(dane=dane,
                          n=10,
                          iteracje=100, 
                          metodaSelekcji='turniejowa',
                          metodaKrzyzowania='OX',
                          prawdopodobienstwoMutacji=0.2)
print("najlepsza droga: ", wynik[0], ", długość: ", wynik[-1])
print("tabela najlepszych wyników w poszczególnych iteracjach: ")
tabela

[[  656.19509294     0.          1870.44379761 ...  5179.89034633
   6492.89211369  6165.48457139]
 [ 1556.30331234  1870.44379761     0.         ...  3612.79946856
   6384.2167883   8034.20388091]
 [ 1352.7808396   1995.73946195  1411.20090703 ...  4858.1708492
   5009.53490855  7803.1019473 ]
 ...
 [ 5096.08477167  5179.89034633  3612.79946856 ...     0.
   9312.5678521  10968.5958992 ]
 [ 5911.4492301   6492.89211369  6384.2167883  ...  9312.5678521
      0.         10006.97516735]
 [ 6612.          6165.48457139  8034.20388091 ... 10968.5958992
  10006.97516735     0.        ]]


100%|██████████| 100/100 [00:07<00:00, 12.55it/s]


najlepsza droga:  [ 18   6   9 111  88 115 101 102  45  56  52  77  94 100  39  15 121  79
  67  68  46 110  20  28  36  29 109 106 119  55  81 125  51  92  66 105
  97  98  58  11  80  57  43  60 107  82  27  70  89  30  83  17   2   0
   4  95  93 117  75  40 122  71  13 112   7  85  87 108  74  49  53  33
  35 123  47  48  21  34  16  22  63 113  50  19  54 120   1   5  84  90
 103  62  44  41  38  73  72  12  76  26  10  78  65  86  91  23  37 114
  99  14 118 124  31 104   8   3  64  59  69 116  24  96  25  61  32  42] , długość:  495224.31943863205
tabela najlepszych wyników w poszczególnych iteracjach: 


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,"[18, 6, 9, 111, 114, 115, 42, 102, 39, 56, 52,...","[18, 6, 9, 111, 114, 115, 42, 102, 39, 56, 52,...","[18, 6, 9, 111, 114, 115, 42, 102, 39, 56, 52,...","[18, 6, 9, 111, 114, 115, 42, 102, 39, 56, 52,...","[18, 6, 9, 111, 114, 115, 42, 102, 39, 56, 52,...","[18, 6, 9, 111, 114, 115, 42, 102, 39, 56, 52,...","[18, 6, 9, 111, 114, 115, 42, 102, 39, 56, 52,...","[18, 6, 9, 111, 114, 115, 42, 102, 39, 56, 52,...","[18, 6, 9, 111, 114, 115, 42, 102, 39, 56, 52,...","[18, 6, 9, 111, 114, 115, 42, 102, 39, 56, 52,...",...,"[18, 6, 9, 111, 88, 115, 101, 102, 45, 56, 52,...","[18, 6, 9, 111, 88, 115, 101, 102, 45, 56, 52,...","[18, 6, 9, 111, 88, 115, 101, 102, 45, 56, 52,...","[18, 6, 9, 111, 88, 115, 101, 102, 45, 56, 52,...","[18, 6, 9, 111, 88, 115, 101, 102, 45, 56, 52,...","[18, 6, 9, 111, 88, 115, 101, 102, 45, 56, 52,...","[18, 6, 9, 111, 88, 115, 101, 102, 45, 56, 52,...","[18, 6, 9, 111, 88, 115, 101, 102, 45, 56, 52,...","[18, 6, 9, 111, 88, 115, 101, 102, 45, 56, 52,...","[18, 6, 9, 111, 88, 115, 101, 102, 45, 56, 52,..."
1,586340.331879,586340.331879,585514.935329,582892.084066,576981.761516,576981.761516,576981.761516,575659.250195,575659.250195,575659.250195,...,499314.752283,499314.752283,499314.752283,498697.958642,498697.958642,498697.958642,498697.958642,495224.319439,495224.319439,495224.319439
